In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install rdkit-pypi
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [ ]:
logP_data = pd.read_csv('/kaggle/input/chemical-structure-and-logp/logP_dataset.csv', 
                        names=['smiles', 'logP'])

print(logP_data.shape)
logP_data.head()

In [ ]:
# distribution of logP, target value
logP_data.logP.hist(bins=50)

In [ ]:
# generate mol object from smiles

df = logP_data.copy()

df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))
df.head(2)

In [ ]:
# first 15 mol's images
mols = df['mol'][:15]
Draw.MolsToGridImage(mols, molsPerRow=5, useSVG=True, legends=list(df['smiles'][:15].values))

In [ ]:
df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x)) # add H atom
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms()) # atom's number in mol
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms()) # exclude H atom
df.head(2)

In [ ]:
# Re-drawing

Draw.MolsToGridImage((df.mol[0], df.mol[1], df.mol[2]), subImgSize=(200,200))

In [ ]:
# indicated atom's number in mol

def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

number_of_atoms(['C','O', 'N', 'Cl', 'S', 'P'], df)

df.head(20)

In [ ]:
# Linear Regression model -1

# feature and target

from sklearn.model_selection import train_test_split

feature = df.drop(columns=['smiles', 'mol', 'logP'])
target = df['logP'].values

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
def show_reg_result(y_test, y_pred, N=75):
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    R2 = r2_score(y_test, y_pred)
    max_err = np.abs(y_test - y_pred).max()

    print('R2:', round(R2, 4))
    print('MAE:', round(mae, 4))
    print('RMSE:', round(rmse,4))
    print('Max error:', round(max_err, 4))

    # ploting with real and pred values

    if N > 0: 
        plt.figure(figsize=(12, 4)) 
        plt.plot(y_pred[:N], ".b-", label="prediction", linewidth=1.0) 
        plt.plot(y_test[:N], '.r-', label="actual", linewidth=1.0) 
        plt.legend() 
        plt.ylabel('logP') 
        plt.show()

In [ ]:
# Linear Regression Model-1

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
show_reg_result(y_test, y_pred)

In [ ]:
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))
df.head()

In [ ]:
feature = df.drop(columns=['smiles', 'mol', 'logP'])
target = df['logP'].values

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=.2, random_state=2310)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
show_reg_result(y_test, y_pred,75)

In [ ]:
X_train.columns

In [ ]:
y_train

In [ ]:
lr.coef_

In [ ]:
lr.intercept_ 

In [ ]:
# feature importances

def plot_feature_weight(feature, weight):
    plt.figure(figsize=(5,3)) 
    W = pd.DataFrame({'feature':feature,'weight':weight})
    W.sort_values('weight', inplace=True)
    plt.barh(W.feature, W.weight)
    
plot_feature_weight(feature.columns, lr.coef_)

In [ ]:
# to find max_depth
# Decision tree regression model
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

res = []
for depth in range(1,50,2):
    dtr = DecisionTreeRegressor(max_depth=depth)
    dtr.fit(X_train, y_train)
    res.append((depth, dtr.score(X_test, y_test).round(4)))
res[:5]

In [ ]:
df_res = pd.DataFrame(res, columns=['depth', 'score']).set_index('depth')
df_res[:5]

In [ ]:
df_res.plot()

In [ ]:
df_res.idxmax(), df_res.max()

In [ ]:
# ploting with real and pred values

dtr = DecisionTreeRegressor(max_depth=21)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
show_reg_result(y_test, y_pred)

In [ ]:
# feature importances

plot_feature_weight(feature.columns, dtr.feature_importances_)

In [ ]:
# Random forest
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
show_reg_result(y_test, y_pred)

In [ ]:
# feature importance

plot_feature_weight(feature.columns, rfr.feature_importances_)

In [ ]:
from sklearn import tree
import matplotlib
plt.figure(figsize=(26,12))

tree.plot_tree(dtr, fontsize=14,
              feature_names=feature.columns,
              filled=True,
              impurity=True,
              max_depth=3)
plt.show()

In [ ]:
def mol_2_fp(mol):
  fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024)
  return fp

In [ ]:
list_fp = df['mol'].apply(mol_2_fp)
ecfp = np.vstack(list_fp)
print(ecfp.shape)
ecfp[:3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ecfp, target, test_size=.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# linear regression model

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
show_reg_result(y_test, y_pred, 0)

In [ ]:
# Light GBM model

from lightgbm import LGBMRegressor
lgbmr = LGBMRegressor()
lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)
show_reg_result(y_test, y_pred,0)

In [ ]:
# ploting with real and pred values

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)
show_reg_result(y_test, y_pred)